In [9]:
using DataFrames
using Chemfiles

home = "/home/german/labo/18/egfr/modos/"
rehome = "/home/german/labo/18/egfr/gram/"
aa = 277

277

In [10]:
function tognm(vtor_anm)
    vtor_gnm = Array{Float64}(convert(Int64, length(vtor_anm)/3));
    vtor_anm =  vtor_anm.^2
    for i=1:convert(Int64, length(vtor_anm)/3)
        vtor_gnm[i] = sqrt(vtor_anm[i*3-2] + vtor_anm[i*3-1] + vtor_anm[i*3])
    end
    return vtor_gnm
end

tognm (generic function with 1 method)

In [11]:
function qgrammian0(mtx_1, mtx_2, weight_1 = 0)
    # Projects mtx_1 onto mtx_2 and gets the mean of the elements in the diagonal.
    if size(mtx_1)[1] != size(mtx_2)[1]
        println("Length of matrices columns must be the same. Aborting.")
        return 0
    end
    
    mtx_g = transpose(mtx_1) * mtx_2 * transpose(mtx_2) * mtx_1
      
    if weight_1 != 0
        # Weights were specified
        if length(weight_1) != size(mtx_1)[2]
            println("There must be as many weights as columns. Aborting.")
            return 0
        end
        weighted = diag(mtx_g) .* weight_1
        zeta = sum(weighted)
    else
        zeta = mean(diag(mtx_g))   
    end
    
    
    return zeta
end

qgrammian0 (generic function with 2 methods)

In [12]:
# Leo nombres de structuras
tmp = readdlm(string(rehome, "/pdbs.list"))
pdbs_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));
aa = 277
aa_3 = aa * 3
aa_36 = aa_3 - 6
structs = length(pdbs_list)

# Leo activas e inactivas
tmp = readdlm(string(rehome, "/am.list"))
am_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));

tmp = readdlm(string(rehome, "/im.list"))
im_list = convert(Array{String, 1}, reshape(tmp, length(tmp)));

In [13]:
# Leo nonmissing residuos de pocket
tmp = readdlm(string(rehome, "nonmis_aa_pocket"))
nonmis_aa_pocket = convert(Array{Int64, 1}, reshape(tmp, length(tmp)))

aa_pocket_nonmis = length(nonmis_aa_pocket)
aa_3_pocket_nonmis = aa_pocket_nonmis * 3

nonmis_aa_pocket_3 = Array{Int64}(aa_3_pocket_nonmis)
for i = 1:aa_pocket_nonmis
    nonmis_aa_pocket_3[3*i - 2] = nonmis_aa_pocket[i] * 3
    nonmis_aa_pocket_3[3*i - 1] = nonmis_aa_pocket[i] * 3 + 1
    nonmis_aa_pocket_3[3*i] = nonmis_aa_pocket[i] * 3 + 2
end

# Leo todos los residuos de pocket
tmp = readdlm(string(rehome, "aa_pocket"))
aa_pocket = convert(Array{Int64, 1}, reshape(tmp, length(tmp)))

aa_pocket_nonmis = length(aa_pocket)
aa_3_pocket_nonmis = aa_pocket_nonmis * 3

aa_pocket_3 = Array{Int64}(aa_3_pocket_nonmis)
for i = 1:aa_pocket_nonmis
    aa_pocket_3[3*i - 2] = aa_pocket[i] * 3
    aa_pocket_3[3*i - 1] = aa_pocket[i] * 3 + 1
    aa_pocket_3[3*i] = aa_pocket[i] * 3 + 2
end

## Versión con los residuos nonmissing del pocket

In [5]:
am_pesos = Array{Float64, 2}(aa_36, length(am_list))

for i = 1:length(am_list)
    each = am_list[i]

    modos = readdlm(string(rehome, each, "/", "modos_", each))[:, 1:aa_36]
    am_pesos[:, i] = [ norm(modos[nonmis_aa_pocket_3, k]) for k = 1:aa_36 ]
    am_pesos[:, i] = am_pesos[:, i] ./ sum(am_pesos[:, i])
end

In [6]:
im_pesos = Array{Float64, 2}(aa_36, length(im_list))

for i = 1:length(im_list)
    each = im_list[i]

    modos = readdlm(string(rehome, each, "/", "modos_", each))[:, 1:aa_36]
    im_pesos[:, i] = [ norm(modos[nonmis_aa_pocket_3, k]) for k = 1:aa_36 ]
    im_pesos[:, i] = im_pesos[:, i] ./ sum(im_pesos[:, i])
end

## Versión con todos los residuos del pocket

In [14]:
am_pesos = Array{Float64, 2}(aa_36, length(am_list))

for i = 1:length(am_list)
    each = am_list[i]

    modos = readdlm(string(rehome, each, "/", "modos_", each))[:, 1:aa_36]
    am_pesos[:, i] = [ norm(modos[aa_pocket_3, k]) for k = 1:aa_36 ]
    am_pesos[:, i] = am_pesos[:, i] ./ sum(am_pesos[:, i])
end

In [15]:
im_pesos = Array{Float64, 2}(aa_36, length(im_list))

for i = 1:length(im_list)
    each = im_list[i]

    modos = readdlm(string(rehome, each, "/", "modos_", each))[:, 1:aa_36]
    im_pesos[:, i] = [ norm(modos[aa_pocket_3, k]) for k = 1:aa_36 ]
    im_pesos[:, i] = im_pesos[:, i] ./ sum(im_pesos[:, i])
end

## Obtengo los zeta vals

In [16]:
am_zeta = zeros(length(am_list) - 1, length(am_list) - 1)

for i = 1:length(am_list)
    each = am_list[i]
    for j = i+1:length(am_list)
        cada = am_list[j]

        modos_1 = readdlm(string(rehome, each, "/", "modos_", each))[:, 1:aa_36]
        modos_2 = readdlm(string(rehome, each, "/", each, "_", cada, "_ord"))[:, 1:aa_36]

        am_zeta[j-1, i] = qgrammian0(modos_1, modos_2, am_pesos[:, j])
    end
end

In [17]:
im_zeta = zeros(length(im_list) - 1, length(im_list) - 1)

for i = 1:length(im_list)
    each = im_list[i]
    for j = i+1:length(im_list)
        cada = im_list[j]

        modos_1 = readdlm(string(rehome, each, "/", "modos_", each))[:, 1:aa_36]
        modos_2 = readdlm(string(rehome, each, "/", each, "_", cada, "_ord"))[:, 1:aa_36]

        im_zeta[j-1, i] = qgrammian0(modos_1, modos_2, im_pesos[:, j])
    end
end

In [20]:
im_pesos

825×15 Array{Float64,2}:
 0.000821179  0.00139029   0.000878193  …  0.00075967   0.000320683
 0.00101854   0.00156213   0.00121529      0.00147416   0.000583021
 0.00108597   0.000987874  0.000848967     0.00115515   0.000340568
 0.000855051  0.00146671   0.000979457     0.00116796   0.000763678
 0.000842752  0.00218525   0.000483899     0.000970831  0.00039242 
 0.000829664  0.0013982    0.00100791   …  0.000904448  0.000924051
 0.000796267  0.0012323    0.000918887     0.00160569   0.00131339 
 0.000969446  0.00134088   0.000541578     0.00129438   0.000982642
 0.00128948   0.00116079   0.00157887      0.0010332    0.00130366 
 0.00135483   0.000977449  0.00158905      0.00125061   0.00157859 
 0.000784397  0.00114147   0.00141863   …  0.00181325   0.000718203
 0.000985989  0.00106184   0.000975283     0.00136798   0.0012242  
 0.00128666   0.00105374   0.00130973      0.000780017  0.00134622 
 ⋮                                      ⋱                          
 0.00116067   0.0012921

In [18]:
am_zeta

31×31 Array{Float64,2}:
 0.999998  0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.999997  0.999996  0.0       0.0          0.0       0.0       0.0     
 0.999995  0.999995  0.999997  0.0          0.0       0.0       0.0     
 0.999994  0.999994  0.999992  0.999994     0.0       0.0       0.0     
 0.99996   0.99996   0.999963  0.999961     0.0       0.0       0.0     
 0.999996  0.999996  0.999995  0.999996  …  0.0       0.0       0.0     
 0.999946  0.999945  0.999948  0.999944     0.0       0.0       0.0     
 0.999996  0.999995  0.999996  0.999996     0.0       0.0       0.0     
 0.999939  0.999941  0.999942  0.99994      0.0       0.0       0.0     
 0.999967  0.999968  0.999965  0.999962     0.0       0.0       0.0     
 0.999992  0.999992  0.999991  0.999992  …  0.0       0.0       0.0     
 0.999994  0.999994  0.999996  0.999995     0.0       0.0       0.0     
 0.999995  0.999993  0.999997  0.999994     0.0       0.0       0.0     
 ⋮                         

In [19]:
im_zeta

14×14 Array{Float64,2}:
 0.999963  0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.998819  0.999968  0.0       0.0          0.0       0.0       0.0     
 0.999984  0.999964  0.999983  0.0          0.0       0.0       0.0     
 0.999851  0.999812  0.999834  0.99986      0.0       0.0       0.0     
 0.999959  0.999968  0.999958  0.999954     0.0       0.0       0.0     
 0.999815  0.999824  0.999807  0.999805  …  0.0       0.0       0.0     
 0.999933  0.999951  0.999933  0.999922     0.0       0.0       0.0     
 0.999916  0.999912  0.99991   0.999915     0.0       0.0       0.0     
 0.998917  0.999907  0.999921  0.999946     0.0       0.0       0.0     
 0.999969  0.999982  0.999974  0.999972     0.0       0.0       0.0     
 0.999964  0.999972  0.99997   0.999965  …  0.0       0.0       0.0     
 0.999804  0.999855  0.999801  0.999793     0.999784  0.0       0.0     
 0.999846  0.999879  0.999847  0.999831     0.99984   0.999978  0.0     
 0.999955  0.999963  0.9999

In [21]:
writedlm(string(rehome, "am_zeta"), am_zeta)

In [22]:
writedlm(string(rehome, "im_zeta"), im_zeta)